# 準備

## GPUを使えるようにする

1. メニューから `Runtime>Change runtime type` を選択する
2. Hardware acceleratorでGPUを選択する

In [ ]:
!nvidia-smi

## Google driveのマウント

In [ ]:
from google.colab import drive
drive._mount('/content/drive')

## Google Drive上でのディレクトリ作成

コマンドは `% + コマンド` で実現していきます．

Google Driveのルートディレクトリである　`/content/drive/MyDrive`  に移動します．

In [ ]:
%cd /content/drive/MyDrive/

```
MyDrive
|--TDSW_DLWS
    |--VAE
        |--models
```
という構造でディレクトリを作成します．

In [ ]:
%mkdir TDSW_DLWS/AE/ TDSW_DLWS/AE/models

ディレクトリを移動します．

In [ ]:
%cd TDSW_DLWS/AE/models

# 学習

## ライブラリのインポート
必要なライブラリ，モジュールを読み込みます．  
それぞれの機能については実際に使用する段階で説明します．


In [115]:
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.datasets import mnist
import numpy as np
from tensorflow.keras.models import load_model
import matplotlib.pyplot as plt

from tensorflow.keras.callbacks import TensorBoard

## データセットの読み込み＋前処理

### MNIST（画像とラベル）の読み込み
CIFAR-10データセットを読み込みます．
これは下の図のような70,000枚(学習用60,000 + 検証用10,000)の28x28pxのGRAYSCALE画像とそのラベル（どのクラスに属するか）のセットです．

</br>

<img src="https://drive.google.com/uc?id=1G-tfBSJ76-Ug0F-DhGvUUh_Lj10OpgFa" width = 50%>

</br>

MNISTはCIFAR-10などと同じくKerasに同封されているためサクッと読み込めます．

- x_train: 入力する訓練データ
- x_test: テスト用データ
- y_train: x_trainの各観測が属するクラス0 -- 9
- y_test: x_testの各観測が属するクラス0 -- 9
- mnist.load_data(): データを読み込む関数

データセットは訓練用とテスト用に分ける必要があります．

訓練時には見せず，**未知のデータに対して推論できているかどうか確かめる**ためです．

In [116]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

In [ ]:
print(x_train.shape)

In [ ]:
print(y_train.shape)

### x_trainを保存しておく
後ほどTouchDesigner側でのプロットで使用するため，x_trainを保存しておきます．

In [119]:
np.save('mnist_x_train', x_train)
np.save('mnist_x_test', x_test)

### 0 - 255を0.0 - 1.0の32bit floatに変換
ニューラルネットワークの入力は-1.0 -- 1.0で最も機能するため，割り算します．

In [120]:
x_train = x_train.astype('float32') / 255.
x_test = x_test.astype('float32') / 255.

### 平滑化
reshape関数で`[60000, 28, 28]`から`[60000, 784]`に平滑化しておきます．


In [121]:
x_train = x_train.reshape((len(x_train), np.prod(x_train.shape[1:])))
x_test = x_test.reshape((len(x_test), np.prod(x_test.shape[1:])))

## アーキテクチャのセットアップ
Sequentialモデルではなく，より自由度の高いFunctional APIにて設計していきます．  
あまり記述量も変わらないので，最初からFunctional APIで実装していくことをおすすめします．

### Encoder
圧縮された潜在表現を得るEncoderを作ります．

### 入力層
入力層には任意の枚数の画像を渡します．  
shapeは入力の次元数を示しています．  
実際の入力はもう１次元あるのですが，バッチサイズは任意の枚数であるため，指定していません．  

### 全結合層
前の層と密結合したノードを指定の数持てます．  
今回は 784->256->64->16->2 と，次元を圧縮していきます．

### 活性化関数
活性化関数はニューラルネットが複雑な機能を学習し，単なる線形な組み合わせを出力しないように重みなどに掛け合わされて伝搬の具合に変化を与えます．  
今回はゼロ以下をすべてゼロにするReLUを使用します．

In [122]:
encoding_dim = 2

encoder_input_layer = Input(shape=(784,), name='encoder_input')
e = Dense(256, activation='relu')(encoder_input_layer)
e = Dense(64, activation='relu')(e)
e = Dense(16, activation='relu')(e)
encoder_output_layer = Dense(encoding_dim, activation='relu', name='encoder_output')(e)

encoder = Model(encoder_input_layer, encoder_output_layer, name='encoder')

### エンコーダの確認

In [ ]:
encoder.summary()

### Encoder
復元するDecoderを作ります．

### 全結合層
前の層と密結合したノードを指定の数持てます．  
今回は Encoderと逆の2->16->64->256->784 と，次元を復元していきます．

### 活性化関数
活性化関数はニューラルネットが複雑な機能を学習し，単なる線形な組み合わせを出力しないように重みなどに掛け合わされて伝搬の具合に変化を与えます．  
今回はゼロ未満をすべてゼロにするReLUを使用します．  
また，出力層ではsigmoidを使用します．  

In [124]:
decoder_input_layer = Input(shape=(encoding_dim,), name='decoder_input')
d = Dense(16, activation='relu')(decoder_input_layer)
d = Dense(64, activation='relu')(d)
d = Dense(256, activation='relu')(d)
decoder_output_layer = Dense(784, activation='sigmoid', name='decoder_output')(d)

decoder = Model(decoder_input_layer, decoder_output_layer, name='decoder')

In [ ]:
decoder.summary()

### EncoderとDecoderを連結
EncoderとDecoderを同時に訓練するため，連結したネットワークにする必要があります．  
Kerasではどの出力から連結するのか指定します．

In [126]:
ae_input_layer = encoder_input_layer
ae_output_layer = decoder(encoder(encoder_input_layer))

### EncoderとDecoderをあわせたAutoEncoderとして宣言

In [127]:
autoencoder = Model(ae_input_layer, ae_output_layer, name='autoencoder')

### アーキテクチャの確認

In [ ]:
autoencoder.summary()  

## モデルのコンパイル
普段聞くコンパイルは，  

**プログラム言語->コンピュータが実行可能な形式への変換**  

を指しますが，Kerasにおけるコンパイルは

**訓練プロセスを作る**

ことを指します．

### オプティマイザ
訓練を進める上での最適化アルゴリズムです．  
今回はAdam（移動平均で振動を抑制するモーメンタム と 学習率を調整して振動を抑制するRMSPropの組み合わせ）を使用します．  

### 損失関数（Loss）
訓練中に導いたものと正解の差です．  
今回は入力画像と出力画像のピクセル値に対して２値交差エントロピーを使用します．  
２値交差エントロピーは大きく間違った極端な予想に対して誤差を大きく返します．

In [129]:
autoencoder.compile(optimizer='adam', loss='binary_crossentropy')

## Tensorboardの起動

In [ ]:
%load_ext tensorboard
%tensorboard --logdir logs

## モデルの訓練

In [131]:
tf_callback = TensorBoard(log_dir="logs", histogram_freq=1)

### バッチサイズ
各訓練ステップでひとまとまりとして渡す観測数です．  
小さいと細かい特徴を捉えられたり精度が上がる反面，局所的な観測に影響を受けてしまうことがあります．  
大きいと平均化されますが，局所的な観測の影響は少なくなります．  
今回は256にします．

### エポック
ネットワークに全トレーニングデータを渡す回数です．  
今回は100回にしておきます．

### シャッフル
真のとき，各訓練ステップで訓練データからランダムに重複なく取り出します．

In [ ]:
autoencoder.fit(x_train, x_train,
                epochs=100,    
                batch_size=256,
                shuffle=True,
                validation_data=(x_test, x_test),
                callbacks=[tf_callback])

### 訓練したモデルの保存

In [133]:
autoencoder.save('AE.h5')

### 訓練後のEncoderを抽出
訓練後の重みを含んだモデル全体からEncoderまでを取り出します．
入力と出力の層を指定することで抽出できます．

In [69]:
encoder_ae = Model(autoencoder.input, autoencoder.get_layer('encoder').output)

### Encoderのコンパイル

In [70]:
encoder_ae.compile(optimizer='adam', loss='binary_crossentropy')

### Enocoderの保存

In [71]:
encoder_ae.save('AE_encoder.h5')

### 訓練後のDecoderを抽出
訓練後の重みを含んだモデル全体からDecoderまでを取り出します．
入力と出力の層を指定することで抽出できます．

In [72]:
decoder_ae = Model(autoencoder.get_layer('decoder').input, autoencoder.get_layer('decoder').output)

### Decoderのコンパイル

In [73]:
decoder_ae.compile(optimizer='adam', loss='binary_crossentropy')

### Decoderの保存

In [74]:
decoder_ae.save('AE_decoder.h5')

## モデルの検証

### 保存したモデルの読み込み


In [75]:
model = load_model('AE.h5')

### テストセットに対する評価
引数はテスト用のデータセットです．

In [ ]:
model.evaluate(x_test, x_test)

### 画像とともに表示

In [ ]:
decoded_imgs = model.predict(x_test)

n = 10
plt.figure(figsize=(10, 2))
for i in range(n):
    ax = plt.subplot(2, n, i+1)
    plt.imshow(x_test[i].reshape(28, 28))
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)

    ax = plt.subplot(2, n, i+1+n)
    plt.imshow(decoded_imgs[i].reshape(28, 28))
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
plt.show()

# ONNXへ変換

## ONNXとtf2onnxをインストール

In [ ]:
!pip install onnx
!pip install tf2onnx

## 変換用スクリプト使用のためレポジトリをクローン

In [ ]:
!git clone https://github.com/onnx/tensorflow-onnx

## TensorFlow saved modelとして保存
ONNXへの変換はkeras2onnxよりtx2onnx経由のほうが成功します．

In [ ]:
import tensorflow as tf

model = load_model('AE.h5')
tf.saved_model.save(model, 'tmp_model_AE')
model = load_model('AE_encoder.h5')
tf.saved_model.save(model, 'tmp_model_AE_encoder')
model = load_model('AE_decoder.h5')
tf.saved_model.save(model, 'tmp_model_AE_decoder')

## 変換＋保存

In [ ]:
!python -m tensorflow-onnx.tf2onnx.convert --saved-model tmp_model_AE --output "AE.onnx"
!python -m tensorflow-onnx.tf2onnx.convert --saved-model tmp_model_AE_encoder --output "AE_encoder.onnx"
!python -m tensorflow-onnx.tf2onnx.convert --saved-model tmp_model_AE_decoder --output "AE_decoder.onnx"